# Assembling

This notebook is used to assemble a processable h5ad object for subsequent notebooks.

## Fill in input data, output and settings

In [1]:
####################### TEST NAME ###################################
test = 'Esophagus'
################### DEFINING STRATEGY ###############################
from_h5ad = True #option 1
assembling_10_velocity = False #option 2
assembling_10_public = False #option 3
convert_seurat = False #option 4

###################### INPUT DATA ###################################

#For option 1: The path to an existing .h5ad file
h5ad_path = '/home/rstudio/preprocessing/data/anndata/Esophagus.h5ad'
#################### Calc QC Columns ###############################
n_threads = 8

fragments_file = '/home/rstudio/preprocessing/data/bamfiles/fragments_Esophagus.bed'


##################### OUTPUT DATA ###################################
output_dir = '/home/rstudio/processed_data'

## Import modules

In [2]:
# sctoolbox modules 
import sctoolbox.atac_tree as sub_tree
import sctoolbox.creators as cr
import sctoolbox.fragment_length as fragments
# import episcanpy
import episcanpy as epi

## Setup path handling object 

In [3]:
# make an instance of the class
tree = sub_tree.ATAC_tree()
# set processing/output directory
tree.processing_dir = output_dir
# set sample/experiment.. 
tree.run = test

/home/rstudio/processed_data/Esophagus/assembling: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/qc: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/norm_correction: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/clustering: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/annotation: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/complete_report: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/qc/plots: NEWLY SETUP
/home/rstudio/processed_data/Esophagus/assembling/anndata: NEWLY SETUP


## Read in data

### Option 1: Read from .h5ad

In [4]:
if from_h5ad:
    
    adata = epi.read_h5ad(h5ad_path)
    
    #Add information to the infoprocess
    cr.build_infor(adata, "Input_for_assembling", h5ad_path)
    cr.build_infor(adata, "Strategy", "Read from h5ad")

## Inspect adata

In [5]:
display(adata)

AnnData object with n_obs × n_vars = 73652 × 150152
    obs: 'barcode', 'TN', 'UM', 'PP', 'UQ', 'CM', 'file', 'sample'
    var: 'name'
    uns: 'infoprocess', 'color_set'

In [6]:
display(adata.var)

,name
0,b'chr1':10008-10690
1,b'chr1':11179-11381
2,b'chr1':28734-29439
3,b'chr1':29495-29682
4,b'chr1':34673-35029
...,...
150147,b'chrY':57203305-57203488
150148,b'chrY':57206427-57206551
150149,b'chrY':57215321-57215411
150150,b'chrY':57215521-57215837


In [7]:
display(adata.obs)

,barcode,TN,UM,PP,UQ,CM,file,sample
1,AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
2,AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/Esophagus.snap,MB
3,AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
4,AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/Esophagus.snap,MB
5,AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
...,...,...,...,...,...,...,...,...
73648,TTCCATCCTCTTTCGCGTGTAA,9.0,6.0,6.0,6.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
73649,TTCCATCCTCTTTCTGCAGACT,10.0,6.0,6.0,6.0,1.0,/opt/project/data/snap/Esophagus.snap,MB
73650,TTCCATCCTCTTTCTGGCGCAG,17.0,5.0,5.0,5.0,1.0,/opt/project/data/snap/Esophagus.snap,MB
73651,TTCCATCCTCTTTGCCGGAAGG,8.0,7.0,7.0,7.0,0.0,/opt/project/data/snap/Esophagus.snap,MB


## Check for QC related columns

### 1. Check if mean fragment length column exists


In [8]:
mfl_column = fragments.check_mfl(adata)
mfl_column

False

## Calculate missing columns

### 1. calc mean fragment length if missing 

In [9]:
if not mfl_column:
    adata = fragments.add_mfl_fragment(fragments_file, adata, n_threads)
adata.obs

,TN,UM,PP,UQ,CM,file,sample,mean_fragment_length
barcode,,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,89.222222
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,176.681818
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,161.166667
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/Esophagus.snap,MB,144.593087
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,122.666667
...,...,...,...,...,...,...,...,...
TTCCATCCTCTTTCGCGTGTAA,9.0,6.0,6.0,6.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,125.800000
TTCCATCCTCTTTCTGCAGACT,10.0,6.0,6.0,6.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,75.000000
TTCCATCCTCTTTCTGGCGCAG,17.0,5.0,5.0,5.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,77.000000


### 2. Promotor enrichment 

## Inspect adata.obs

In [10]:
adata.obs

,TN,UM,PP,UQ,CM,file,sample,mean_fragment_length
barcode,,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,89.222222
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,176.681818
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,161.166667
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/Esophagus.snap,MB,144.593087
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,122.666667
...,...,...,...,...,...,...,...,...
TTCCATCCTCTTTCGCGTGTAA,9.0,6.0,6.0,6.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,125.800000
TTCCATCCTCTTTCTGCAGACT,10.0,6.0,6.0,6.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,75.000000
TTCCATCCTCTTTCTGGCGCAG,17.0,5.0,5.0,5.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,77.000000


## Save adata to .h5ad

In [11]:
adata_output = tree.assembled_anndata
adata_output

'/home/rstudio/processed_data/Esophagus/assembling/anndata/Esophagus.h5ad'

In [12]:
#Saving the data
cr.build_infor(adata, "Test_number", test)
cr.build_infor(adata, "Anndata_path", output_dir)

adata_output = tree.assembled_anndata
adata.write(filename=adata_output)